In [64]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, BatchNormalization, Dropout, LeakyReLU, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.models import Model
import os

In [59]:
# Dictionary in which we store images' dimensions
# Channels refers to the amount of color channels (RGB)

image_dimensions = {
    'height': 256,
    'width': 256,
    'channels': 3}

# Classifier class

class Classifier: 
    def __init__(self):
        self.model = None
    
    def predict(self, x):
        return self.model.predict(x)
    
    def fit(self, x, y):
        return self.model.train_on_batch(x, y)
    
    def get_accuracy(self, x, y):
        return self.model.test_on_batch(x, y)
    
    def load(self, path):
        self.model.load_weights(path)

In [67]:
# Create a MesoNet class using the Classifier

class Meso4(Classifier):
    def __init__(self, learning_rate = 0.001):
        self.model = self.init_model()
        optimizer = Adam(learning_rate = learning_rate)
        self.model.compile(optimizer = optimizer, 
                           loss = "binary_crossentropy",
                           metrics = ["accuracy"])
    
    def init_model(self):
        x = Input(shape = (image_dimensions['height'],
                            image_dimensions['width'],
                            image_dimensions['channels']))
            
        x1 = Conv2D(8, (3,3), padding = 'same', activation = "relu")(x)
        x1 = BatchNormalization()(x1)
        x1 = MaxPooling2D(pool_size = (2,2), padding = "same")(x1)

        x2 = Conv2D(8, (5,5), padding = 'same', activation = "relu")(x1)
        x2 = BatchNormalization()(x2)
        x2 = MaxPooling2D(pool_size = (2,2), padding = "same")(x2)

        x3 = Conv2D(16, (5,5), padding = 'same', activation = "relu")(x2)
        x3 = BatchNormalization()(x3)
        x3 = MaxPooling2D(pool_size = (2,2), padding = "same")(x3)

        x4 = Conv2D(16, (5,5), padding = 'same', activation = "relu")(x3)
        x4 = BatchNormalization()(x4)
        x4 = MaxPooling2D(pool_size = (4,4), padding = "same")(x4)
        
        y = Flatten()(x4)
        y = Dropout(0.5)(y)
        y = Dense(16)(y)
        y = LeakyReLU(alpha = 0.1)(y)
        y = Dropout(0.5)(y)
        y = Dense(1, activation = 'sigmoid')(y)

        return Model(inputs = x, outputs = y)

In [78]:
# Initiate a MesoNet model with pretrained weigths
meso = Meso4()
meso.load()# mancano i pesi da scaricare
# https://github.com/kiteco/python-youtube-code/tree/master/Deepfake-detection

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


ValueError: File format not supported: filepath=/Users/eleonoramarcassa/Desktop/UniVe/I_anno /Data Analytics and Artificial Intelligence/DeepFake-Detector/python-youtube-code_Deepfake-detection_weights at master · kiteco_python-youtube-code · GitHub_files. Keras 3 only supports V3 `.keras` and `.weights.h5` files, or legacy V1/V2 `.h5` files.

In [77]:
# Prepare image data

# Rescaling pixel values (between 1 and 255) to a range between 0 and 1
dataGenerator = ImageDataGenerator(rescale = 1./255)

# Initiating generator to feed images through the network
generator = dataGenerator.flow_from_directory(
    './dataset big/Train',
    target_size = (256, 256),
    batch_size = 1,
    class_mode = 'binary')

# Checking class assignment 
generator.class_indices

# Rendering image X with label y for MesoNet
X, y = next(iter(generator))

# Evaluating prediction
print(f'Predicted likelihood: {predictions[0][0]:.4f}')

Found 140002 images belonging to 2 classes.


NameError: name 'predictions' is not defined